In [1]:
import requests
import pandas as pd
from pandas import json_normalize
pd.options.display.float_format = '{:.1f}'.format
headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:80.0) Gecko/20100101 Firefox/80.0'}
url = 'https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'
#url = 'https://www.nseindia.com/api/option-chain-indices?symbol=BANKNIFTY'
#url = 'https://www.nseindia.com/api/option-chain-equities?symbol=PETRONET'
    
with requests.session() as s:

    # load cookies:
    s.get('https://www.nseindia.com', headers=headers)

    # get data:
    data = s.get(url, headers=headers).json()

    
spurts = json_normalize(data['filtered']['data'])
#spurtsx = json_normalize(data['filtered']['data'])
spurts.drop([ 'PE.change','expiryDate' , 'PE.strikePrice' , 'PE.expiryDate' , 'CE.strikePrice' , 'CE.underlyingValue', 'PE.identifier' , 'PE.pchangeinOpenInterest' , 'PE.underlyingValue' , 'PE.pChange', 'CE.expiryDate' , 'CE.identifier' ,'CE.expiryDate' ,'CE.totalBuyQuantity', 'PE.underlying' , 'PE.totalTradedVolume', 'CE.pchangeinOpenInterest' , 'PE.totalBuyQuantity' , 'PE.totalSellQuantity' ,'CE.changeinOpenInterest' , 'PE.changeinOpenInterest' , 'CE.pChange', 'CE.bidprice' , 'CE.askPrice', 'CE.askQty' , 'PE.bidprice', 'PE.askQty', 'CE.totalSellQuantity' , 'PE.bidQty', 'PE.askPrice',  'CE.bidQty', 'CE.change' , 'CE.underlying',  'CE.totalTradedVolume' ], axis=1, inplace=True)

spurts.loc[:,'Cpain']=0
spurts.loc[:,'Ppain']=0
for i in range(1,len(spurts)):
    SumOfOpen = spurts.loc[0:i-1,'CE.openInterest'].sum()
    Si = spurts.loc[i,'strikePrice']
    SiPrev = spurts.loc[i-1,'strikePrice']
    spurts.loc[i,'Cpain']= spurts.loc[i-1,'Cpain']+SumOfOpen*(Si-SiPrev)
    spurts.loc[i,'sumofopen']= SumOfOpen
    

for i in range(len(spurts)-2,-1,-1):
    SumOfOpen = spurts.loc[i+1:len(spurts),'PE.openInterest'].sum()
    Si = spurts.loc[i,'strikePrice']
    SiNext = spurts.loc[i+1,'strikePrice']
    spurts.loc[i,'Ppain']= spurts.loc[i+1,'Ppain']+SumOfOpen*(SiNext-Si)
    

spurts.loc[:,'Totalpain']=spurts.loc[:,'Cpain']+spurts.loc[:,'Ppain']
spurts.rename(columns={'PE.impliedVolatility': 'PEIV', 'CE.impliedVolatility': 'CEIV','PE.openInterest': 'PEOI', 'CE.openInterest': 'CEOI','PE.lastPrice': 'PELP'}, inplace=True)
spurts.sort_values(by=['Totalpain'], inplace=True, ascending=True)
Date = (data["records"]["timestamp"])
spurts['Date'] = Date



In [2]:
spca = spurts.head(8)
spca

,strikePrice,CEOI,CEIV,CE.lastPrice,PEOI,PEIV,PELP,Cpain,Ppain,sumofopen,Totalpain,Date
33,17150,22438,20.6,100.3,16346,21.7,166.2,12597300,23193900,114710.0,35791200,06-Dec-2021 12:46:52
34,17200,94556,20.3,79.0,54307,21.5,194.3,19454700,16989800,137148.0,36444500,06-Dec-2021 12:46:52
32,17100,63246,20.9,124.9,63590,21.9,140.0,6861800,30215300,51464.0,37077100,06-Dec-2021 12:46:52
35,17250,21698,20.1,61.1,4200,21.5,227.6,31039900,13501050,231704.0,44540950,06-Dec-2021 12:46:52
31,17050,13910,21.1,152.0,20081,22.2,117.7,4288600,40416200,37554.0,44704800,06-Dec-2021 12:46:52
36,17300,105730,20.1,46.9,22818,21.6,261.9,43710000,10222300,253402.0,53932300,06-Dec-2021 12:46:52
30,17000,25588,21.5,183.5,86599,22.6,98.6,2410900,51621150,11966.0,54032050,06-Dec-2021 12:46:52
29,16950,1820,21.6,216.3,15872,23.0,82.2,1812600,67156050,10146.0,68968650,06-Dec-2021 12:46:52
